In [1]:
from ibapi.client import *
from ibapi.wrapper import *
import datetime
import time
import threading


from ibapi.contract import Contract
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
import psycopg2
from time import sleep
import pandas as pd
port = 7496

class TestApp(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.orderId = 0
        self.list_ = []
        # etf = pd.read_csv("a.csv")
        # self.etfList = list(etf.code)[:]
        etf = pd.read_csv("../st/hk_list.csv")
        self.etfList = list(etf.iloc[:, 2])[:]
        self.etfList = [i.split(".")[0] for i in self.etfList]
        self.index = 0
        self.curCode = self.etfList[self.index]
        # self.tw_maincode_offshore_list = 
    def nextValidId(self, orderId):
        self.orderId = orderId
    def nextId(self):
        self.orderId += 1
        return self.orderId

    def error(self, reqId, errorCode, errorString, advancedOrderReject=""):
        print(f"reqId: {reqId}, errorCode: {errorCode}, errorString: {errorString}, orderReject: {advancedOrderReject}")

    def historicalData(self, reqId, bar):
        bar = bar.to_dict()
        bar['symbol'] = self.curCode
        self.list_.append(bar)
    def historicalDataEnd(self, reqId, start, end):
        self.cancelHistoricalData(reqId)
        self.index += 1
        self.curCode = self.etfList[self.index]
    def loop(self):
        for i in self.etfList:
            mycontract = Contract()
            mycontract.symbol = i
            mycontract.secType = "STK"
            mycontract.exchange = "SEHK"
            mycontract.currency = "HKD"
            nextId = app.nextId()
            app.reqHistoricalData(nextId, mycontract, "20240128 09:00:00 US/Eastern", "1 Y", "1 day", "TRADES", 1, 1, False, [])
app = TestApp()
app.connect("127.0.0.1", port, 0)
threading.Thread(target=app.run).start()
time.sleep(1)

reqId: -1, errorCode: 2104, errorString: Market data farm connection is OK:usfarm.nj, orderReject: 
reqId: -1, errorCode: 2104, errorString: Market data farm connection is OK:hfarm, orderReject: 
reqId: -1, errorCode: 2104, errorString: Market data farm connection is OK:cashfarm, orderReject: 
reqId: -1, errorCode: 2104, errorString: Market data farm connection is OK:usopt, orderReject: 
reqId: -1, errorCode: 2104, errorString: Market data farm connection is OK:usfarm, orderReject: 
reqId: -1, errorCode: 2106, errorString: HMDS data farm connection is OK:euhmds, orderReject: 
reqId: -1, errorCode: 2106, errorString: HMDS data farm connection is OK:fundfarm, orderReject: 
reqId: -1, errorCode: 2106, errorString: HMDS data farm connection is OK:ushmds, orderReject: 
reqId: -1, errorCode: 2158, errorString: Sec-def data farm connection is OK:secdefnj, orderReject: 


reqId: 2, errorCode: 200, errorString: The destination or exchange selected is Invalid. Please review your order's "Destination" field. If using a <br>Directed order, review the exchange selected when creating the order ticket or order row. This may occur when <br>creating stock orders for the overnight session or when creating option orders for the overnight session., orderReject: 
reqId: 3, errorCode: 10314, errorString: End Date/Time: The date, time, or time-zone entered is invalid. The correct format is yyyymmdd hh:mm:ss xx/xxxx where yyyymmdd and xx/xxxx are optional. E.g.: 20031126 15:59:00 US/Eastern  Note that there is a space between the date and time, and between the time and time-zone.  If no date is specified, current date is assumed. If no time-zone is specified, local time-zone is assumed(deprecated).  You can also provide yyyymmddd-hh:mm:ss time is in UTC. Note that there is a dash between the date and time in UTC notation., orderReject: 
reqId: 5, errorCode: 10314, erro

In [8]:
mycontract = Contract()
mycontract.symbol = "1101"
mycontract.secType = "STK"
mycontract.exchange = "SEHK"
mycontract.currency = "HKD"
app.reqHistoricalData(app.nextId(), mycontract, "20250101 09:00:00 US/Eastern", "1 W", "1 day", "TRADES", 1, 1, False, [])


reqId: 8, errorCode: 504, errorString: Not connected, orderReject: 


In [ ]:
app.loop()  

In [ ]:
te = pd.DataFrame(app.list_)
k = list(set(te['symbol']))
len(te), len(k)
te_f = te.drop_duplicates(subset=["symbol", "date"])
te_f = te_f.iloc[:, [0,1, 2, 3, 4, 5, -1]]
te_f['date'] = pd.to_datetime(te_f['date'])
# te_f.to_parquet("2025_v1_tickers_test.parquet")
len(te), len(te_f)

In [ ]:
import psycopg2 # type: ignore
import matplotlib.pyplot as plt # type: ignore
import pandas as pd # type: ignore
from tqdm import tqdm # type: ignore
import numpy as np # type: ignore
config = {
    "host": "10.188.200.16",
    "dbname": "us",
    "user": "jimmy",
    "password": "123123"
}
conn = psycopg2.connect(**config)
cursor = conn.cursor()

# # price
# sqlStr = """
# CREATE TABLE price_ib (
#     da DATE NOT NULL,                 
#     op DOUBLE PRECISION NOT NULL,     
#     hi DOUBLE PRECISION NOT NULL,     
#     lo DOUBLE PRECISION NOT NULL,     
#     cl DOUBLE PRECISION NOT NULL,     
#     vol DOUBLE PRECISION NOT NULL,    
#     code CHAR(50) NOT NULL,           
#     PRIMARY KEY (code, da),           
#     CHECK (op >= 0),                  
#     CHECK (hi >= 0),                  
#     CHECK (lo >= 0),                  
#     CHECK (cl >= 0),                  
#     CHECK (vol >= 0)                  
# );
# """
# cursor.execute(sqlStr)
# conn.commit()
# print("database created")
sqlStr = f"""
    INSERT INTO price_ib (da, op, hi, lo, cl, vol, code)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
"""
data_to_insert = [tuple(row) for row in te_f.iloc[:, :].itertuples(index=False, name=None)]

try:
    for row in tqdm(data_to_insert, desc="Inserting rows"):
        cursor.execute(sqlStr, row)
    
    # Commit the transaction
    conn.commit()
    print("Data inserted successfully!")
except Exception as e:
    # Rollback in case of error
    conn.rollback()
    print(f"Error: {e}")

In [ ]:
# test for missed symbols
k = list(set(te['symbol']))
for i in app.etfList[:]:
    if i not in k: print(i)

In [2]:
mycontract = Contract()
mycontract.symbol = "1101"
mycontract.secType = "STK"
mycontract.exchange = "HKEX"
mycontract.currency = "HKD"
app.reqHistoricalData(app.nextId(), mycontract, "20250120 09:00:00 US/Eastern", "1 Y", "1 day", "TRADES", 1, 1, False, [])
